"""
-------------------------------------------------------------------------------
MOVIE MAKER SCRIPT (Image Sequence to Video)
-------------------------------------------------------------------------------
Simple program to convert an inage sequence to a video<br>
Author: Brian Willis<br>
AI programming support: Gemini 3 pro<br>
Contact: Willis77019@gmail.com<br>
License:  MIT License (unrestricted: Use at your own risk)<br>

DESCRIPTION:
This simple script is a replacement for Microsoft's discontinued Movie Maker. 
It allows you to convert a folder of sequential image files into a single high-quality  movie file (MP4). It is particularly useful for:
- Geological model outputs (simulation time-steps).
- Time-lapse photography (fieldwork or lab experiments).
- Drone flight time laspe image sequences.

PREREQUISITES:
1. Python Libraries:
   - You must install the FFmpeg plugin for imageio.
   - Run this in your terminal/command prompt:
     pip install imageio[ffmpeg]

PERFORMANCE NOTE:
- This script is designed for convenience and ease of use within Python.
- For massive datasets (e.g., 10,000+ 4K frames), the native FFmpeg command-line tool is faster, but this script handles the complexity of file sorting and encoding arguments for you.

CONFIGURATION:
- Set 'FPS' (Frames Per Second) in the script to control speed.
  - 30 FPS = Smooth video.
  - 0.5 FPS = Slideshow (1 image every 2 seconds).
-------------------------------------------------------------------------------
"""


In [1]:
"""
-------------------------------------------------------------------------------
MOVIE MAKER SCRIPT (Sequence to Video)
-------------------------------------------------------------------------------
Author:       Brian Willis (Enabled by Gemini AI)
Contact:      Willis77019@gmail.com
License:      MIT License 

DESCRIPTION:
This script converts a folder of sequential images into an MP4 video.
It automatically handles the "Odd Number" crash by cropping 1 pixel if needed.

PREREQUISITES:
- pip install imageio[ffmpeg]
-------------------------------------------------------------------------------
"""

import os
import glob
import imageio.v2 as imageio # <--- Fixes the "Deprecation" warning
import tkinter as tk
from tkinter import filedialog

# ================= USER CONFIGURATION =================
FPS = 30       # Frames Per Second
QUALITY = 8    # Video Quality (0-10)
# ======================================================

def main():
    print("--- SEQUENCE TO VIDEO STARTING ---")

    # 1. Select Folder
    root = tk.Tk()
    root.withdraw()
    print("Please select your INPUT folder containing image frames...")
    INPUT_FOLDER = filedialog.askdirectory(title="Select Folder With Image Sequence")
    
    if not INPUT_FOLDER: return
    
    # 2. Setup Output
    folder_name = os.path.basename(INPUT_FOLDER)
    OUTPUT_FILE = os.path.join(INPUT_FOLDER, f"{folder_name}_Video.mp4")

    # 3. Gather files
    extensions = ['*.JPG', '*.jpg', '*.PNG', '*.png', '*.JPEG', '*.jpeg']
    files = []
    for ext in extensions:
        files.extend(glob.glob(os.path.join(INPUT_FOLDER, ext)))
    
    files.sort() # Ensure correct order
    
    if not files:
        print(f"No images found in {INPUT_FOLDER}")
        return

    # 4. Check Dimensions (The "Broken Pipe" Fix)
    # Read the first image to check for odd numbers
    first_img = imageio.imread(files[0])
    height, width, _ = first_img.shape
    
    print(f"Found {len(files)} frames. Source Resolution: {width}x{height}")
    
    # Calculate safe dimensions (Must be divisible by 2)
    safe_width = width
    safe_height = height
    
    if width % 2 != 0: safe_width -= 1
    if height % 2 != 0: safe_height -= 1
    
    if safe_width != width or safe_height != height:
        print(f"[WARNING] Odd resolution detected!")
        print(f"  FFmpeg requires even numbers. Cropping video to: {safe_width}x{safe_height}")
    
    # 5. Create Video
    print(f"Stitching video at {FPS} fps...")
    
    try:
        # We explicitly set macro_block_size=None because we are handling the sizing manually
        with imageio.get_writer(OUTPUT_FILE, fps=FPS, quality=QUALITY, macro_block_size=None) as writer:
            for i, filename in enumerate(files):
                if i % 50 == 0:
                    print(f"  Processing frame {i}/{len(files)}...", end='\r')
                
                image = imageio.imread(filename)
                
                # CROP IF NEEDED (To satisfy the Even-Number Rule)
                # Slicing syntax: image[0:safe_height, 0:safe_width, :]
                if safe_width != width or safe_height != height:
                    image = image[:safe_height, :safe_width, :]
                
                writer.append_data(image)

        print(f"\n[DONE] Video saved to: {OUTPUT_FILE}")
        
    except Exception as e:
        print(f"\n[CRITICAL ERROR] {e}")
        print("Tip: If you see 'Broken Pipe', it usually means FFmpeg crashed.")

if __name__ == "__main__":
    main()

--- SEQUENCE TO VIDEO STARTING ---
Please select your INPUT folder containing image frames...
No images found in C:/Users/Willi/Desktop/HDR TST/Stop


In [3]:
pip install imageio[ffmpeg]